In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel,LinearModel
from glob import glob
from lmfit import models

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [15]:
%matplotlib qt 

#Rutherfordstreuung

##Rutherfordsteuuformel
<br>
$\huge \huge \frac{d\sigma}{d\Omega} = 1.3 \cdot 10^{-3} \left( \frac{Z_1 Z_2}{E_0} \right) \frac{1}{sin^4\left(\frac{\theta}{2}\right)}$

<ul>
    <li>$\sigma$: Differentielle WQ
    <li>$\Theta$: Winkel
    <li>$Z_i$: Kernlagundszahl
    <li>E: Energie
</ul>   

In [3]:
def rutherfordsteuung(theta,Z_1,Z_2,E,b):
    e = 1.602 * 10**-19
    return (Z_1*Z_2*e**2)/(4*E*np.sin((3.14*theta-b)/360)**4)

# Energieverlust von $\alpha$-Strahlung in Luft

# Energie-Kanal kalibration

In [4]:
cd /home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/

/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib


In [5]:
data = np.array(pd.read_csv("energiekalpib_30Torr.csv",skiprows=15,delimiter="\t")).T

In [6]:
plt.errorbar(data[0],data[1],np.sqrt(data[1]))
plt.show()

## Händische schätzung für die Peaks

    - 1: x=250, y=180
    - 2: x=320, y=266
    - 3: x=373, y=158
    - 4: x=551, y=159
    
Mulitgauss mit den Werten konfigurieren und Fehler berechnen

In [7]:
err = np.sqrt(data[1][200:612])
err = np.where(err == 0, 1, err)

In [8]:
y = data[1][200:612]
x = data[0][200:612]


g_1 = GaussianModel(prefix="g_1")
pars = g_1.guess(y,x=x)

pars['g_1center'].set(260, min=240, max=275)
pars['g_1sigma'].set(15, min=3,max=22)
pars['g_1amplitude'].set(10000, min=90)

g_2 = GaussianModel(prefix="g_2")
pars.update( g_2.make_params())

pars['g_2center'].set(320, min=300, max=345)
pars['g_2sigma'].set(15, min=3)
pars['g_2amplitude'].set(20000, min=1)


g_3 = GaussianModel(prefix="g_3")
pars.update( g_3.make_params())

pars['g_3center'].set(373, min=350, max=400)
pars['g_3sigma'].set(15, min=3,max=20)
pars['g_3amplitude'].set(10000, min=1)

g_4 = GaussianModel(prefix="g_4")
pars.update( g_4.make_params())

pars['g_4center'].set(551, min=530, max=570)
pars['g_4sigma'].set(15, min=3)
pars['g_4amplitude'].set(20000, min=1)



mod = g_1 + g_2 + g_3 + g_4


out = mod.fit(y,pars,x=x,weights=1/err)

In [9]:
print(out.fit_report())

[[Model]]
    (((Model(gaussian, prefix='g_1') + Model(gaussian, prefix='g_2')) + Model(gaussian, prefix='g_3')) + Model(gaussian, prefix='g_4'))
[[Fit Statistics]]
    # function evals   = 166
    # data points      = 412
    # variables        = 12
    chi-square         = 686.653
    reduced chi-square = 1.717
[[Variables]]
    g_1amplitude:   8700.66368 +/- 308.8273 (3.55%) (init= 10000)
    g_1sigma:       22         +/- 0.251317 (1.14%) (init= 15)
    g_1center:      255.238187 +/- 0.978165 (0.38%) (init= 260)
    g_1fwhm:        51.8060400 +/- 0.591806 (1.14%)  == '2.3548200*g_1sigma'
    g_2amplitude:   12483.3775 +/- 459.5780 (3.68%) (init= 20000)
    g_2sigma:       19.4956985 +/- 0.752448 (3.86%) (init= 15)
    g_2center:      315.461920 +/- 0.556976 (0.18%) (init= 320)
    g_2fwhm:        45.9088608 +/- 1.771881 (3.86%)  == '2.3548200*g_2sigma'
    g_3sigma:       18.5261938 +/- 0.488227 (2.64%) (init= 15)
    g_3amplitude:   7156.31891 +/- 243.6603 (3.40%) (init= 10000)
  

In [10]:
plt.errorbar(data[0][200:612],data[1][200:612],np.sqrt(data[1][200:612]),fmt='. b',label="Messdaten")
plt.plot(data[0][200:612],out.best_fit, '-r',label="Muli-Gauss-Fit")
plt.xlabel("Kanal",fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best', fontsize=42)
plt.show()

## Mittelwerte den Energien zu ordnen
    - Peak  Energie  Kanal
    -   1    4,871   255+1
    -   2    5.49    315+0.6
    -   3    6.00    372+0.8
    -   4    7.69    549+0.3

In [11]:
eich_kanal = np.array([255,315,372,549])
eich_energie = np.array([4.871,5.49,6.00,7.69])
eich_errx = np.array([1,0.6,0.8,0.3])

In [12]:
mod_eich = LinearModel()
guess = mod_eich.guess(eich_energie,x=eich_kanal)
out_eich = mod_eich.fit(eich_energie,guess,x=eich_kanal)
print(out_eich.fit_report())
out_eich.plot()
plt.legend(loc='best')
plt.show()

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 3
    # data points      = 4
    # variables        = 2
    chi-square         = 0.001
    reduced chi-square = 0.001
[[Variables]]
    slope:       0.00953604 +/- 0.000111 (1.16%) (init= 0.009536043)
    intercept:   2.45818986 +/- 0.042991 (1.75%) (init= 2.45819)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.959 


In [13]:
plt.errorbar(eich_kanal,eich_energie,xerr=eich_errx,fmt='.r',label="Messdaten")
plt.plot(eich_kanal,out_eich.best_fit,label="linearer Fit")
plt.xlabel("Kanal",fontsize=42)
plt.ylabel('Energie [MeV]',fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.xlim([240,560])
plt.ylim([4.7,7.9])
plt.legend(loc="best",fontsize=42)
plt.show()

# Fitten der Mulitgauß bei unterschiedlichen Drücken

In [15]:
y = data[1][200:612]
x = data[0][200:612]


g_1 = GaussianModel(prefix="g_1")
pars = g_1.guess(y,x=x)

pars['g_1center'].set(181)
pars['g_1sigma'].set(15, min=3)
pars['g_1amplitude'].set(10000, min=90)

g_2 = GaussianModel(prefix="g_2")
pars.update( g_2.make_params())

pars['g_2center'].set(239)
pars['g_2sigma'].set(15, min=3)
pars['g_2amplitude'].set(10000, min=1)


g_3 = GaussianModel(prefix="g_3")
pars.update( g_3.make_params())

pars['g_3center'].set(292)
pars['g_3sigma'].set(15, min=3)
pars['g_3amplitude'].set(10000, min=1)

g_4 = GaussianModel(prefix="g_4")
pars.update( g_4.make_params())

pars['g_4center'].set(457)
pars['g_4sigma'].set(15, min=3)
pars['g_4amplitude'].set(10000, min=1)



mod = g_1 + g_2 + g_3 + g_4

In [16]:
i = liste[0]
out_2 = mod.fit(i[1][45:-5],pars,x=i[0][45:-5])
out_2.plot()
plt.show()

NameError: name 'liste' is not defined

In [17]:
print(out_2.values['g_1center'])
print(out_2.values['g_2center'])
print(out_2.values['g_3center'])
print(out_2.values['g_4center'])

NameError: name 'out_2' is not defined

In [18]:
liste_fit_params_4.append([])
liste_fit_params_4[-1].append(out_2.best_values["g_4center"])
liste_fit_params_4[-1].append(out_2.best_values["g_4amplitude"])
liste_fit_params_4[-1].append(out_2.best_values["g_4sigma"])

NameError: name 'liste_fit_params_4' is not defined

# Beth-Bloch 

In [19]:
np_liste_fit_1 = np.loadtxt("/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/paras_1").T
np_liste_fit_2 = np.loadtxt("/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/paras_2").T
np_liste_fit_3 = np.loadtxt("/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/paras_3").T
np_liste_fit_4 = np.loadtxt("/home/frederik/Dokumente/FP/Rutherford/MessdatenRutherford/kalib/paras_4").T

In [20]:
delta_mu_1 = np.array([0.5,0.4,0.4,0.4,0.4,5.2])
delta_mu_2 = np.array([0.2,0.2,0.2,0.2,0.2,0.3,0.2,0.3,2.5])
delta_mu_3 = np.array([0.3,0.3,0.3,0.3,0.3,0.3,0.4,0.3,0.6,0.2,1.5,13.2])
delta_mu_4 = np.array([0.2,0.2,0.2,0.1,0.1,0.1,0.2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.2,1.1])
delta_E_1 = delta_mu_1*0.0095
delta_E_2 = delta_mu_2*0.0095
delta_E_3 = delta_mu_3*0.0095
delta_E_4 = delta_mu_4*0.0095

In [21]:
dE_1 = (np_liste_fit_1[0][:-1] - np_liste_fit_1[0][1:])*0.0095
dE_2 = (np_liste_fit_2[0][:-1] - np_liste_fit_2[0][1:])*0.0095
dE_3 = (np_liste_fit_3[0][:-1] - np_liste_fit_3[0][1:])*0.0095
dE_4 = (np_liste_fit_4[0][:-1] - np_liste_fit_4[0][1:])*0.0095
delta_dE_1 = np.sqrt(delta_E_1[:-1]**2 + delta_E_1[1:]**2)
delta_dE_2 = np.sqrt(delta_E_2[:-1]**2 + delta_E_2[1:]**2)
delta_dE_3 = np.sqrt(delta_E_3[:-1]**2 + delta_E_3[1:]**2)
delta_dE_4 = np.sqrt(delta_E_4[:-1]**2 + delta_E_4[1:]**2)

In [22]:
x = (6/760)*np.arange(100,725,25)
dx = (x[:-1]-x[1:])
delta_dx = np.sqrt(2)*dx/dx*0.5

In [23]:
dE_dx_1 = dE_1/dx[:dE_1.size]
dE_dx_2 = dE_2/dx[:dE_2.size]
dE_dx_3 = dE_3/dx[:dE_3.size]
dE_dx_4 = dE_4/dx[:dE_4.size]
delta_dE_dx_1 = np.sqrt((delta_dE_1/dx[:delta_dE_1.size])**2 + 
                        ((delta_dx[:delta_dE_1.size] * dE_1)/dx[delta_dE_1.size])**2 )/6
delta_dE_dx_2 = np.sqrt((delta_dE_2/dx[:delta_dE_2.size])**2 + 
                        ((delta_dx[:delta_dE_2.size] * dE_2)/dx[delta_dE_2.size])**2 )/6
delta_dE_dx_3 = np.sqrt((delta_dE_3/dx[:delta_dE_3.size])**2 + 
                        ((delta_dx[:delta_dE_3.size] * dE_3)/dx[delta_dE_3.size])**2 )/6
delta_dE_dx_4 = np.sqrt((delta_dE_4/dx[:delta_dE_4.size])**2 + 
                        ((delta_dx[:delta_dE_4.size] * dE_4)/dx[delta_dE_4.size])**2 )/6
print(delta_dE_dx_1)

[ 0.15704129  0.15613131  0.15443626  0.16352066  0.18244823]


In [24]:
E_1 = (np_liste_fit_1[0][0:-1] + np_liste_fit_1[0][1:])*0.0095/2
E_2 = (np_liste_fit_2[0][0:-1] + np_liste_fit_2[0][1:])*0.0095/2
E_3 = (np_liste_fit_3[0][0:-1] + np_liste_fit_3[0][1:])*0.0095/2
E_4 = (np_liste_fit_4[0][0:-1] + np_liste_fit_4[0][1:])*0.0095/2

In [25]:
term_1_1 = (((np_liste_fit_1[0][:-1]*0.01)**2 + (np_liste_fit_1[0][1:]*0.01)**2)*0.0095)/dx[:dE_1.size]**2
term_1_2 = (2*((6/760)*8*dE_1)**2)/dx[:dE_1.size]**4
err_1 = np.sqrt(term_1_1 + term_1_2)

In [60]:
plt.plot(E_1,-dE_dx_1,label='4,871 MeV')
plt.plot(E_2,-dE_dx_2,label='5,49 MeV')
plt.plot(E_3,-dE_dx_3,label='6 MeV')
plt.plot(E_4,-dE_dx_4,label='7,69 MeV')
plt.legend()
plt.show()

## Bethe-Bloch $\sim \frac{A}{\beta} \left[ ln(B \cdot \beta) - \beta \right]$

In [27]:
def bethe_func(x,A,B):
    return (A*np.log(B*x)-x)/x

In [28]:
df_liste_fit_1 = pd.DataFrame(np_liste_fit_1.T)
df_liste_fit_2 = pd.DataFrame(np_liste_fit_2.T)
df_liste_fit_3 = pd.DataFrame(np_liste_fit_3.T)
df_liste_fit_4 = pd.DataFrame(np_liste_fit_4.T)

In [29]:
bethe_fit = models.Model(bethe_func)
params = bethe_fit.make_params(A=1,B=1,C=0)

In [30]:
out_bethe_1 = bethe_fit.fit(-dE_dx_1,params,x=E_1,weights= 1/delta_dE_dx_1)
print(out_bethe_1.fit_report())
out_bethe_1.plot()
plt.errorbar(E_1,-dE_dx_1,delta_dE_dx_1,fmt='. ',label='shit')
plt.legend(loc='best')
plt.show()

[[Model]]
    Model(bethe_func)
[[Fit Statistics]]
    # function evals   = 13
    # data points      = 5
    # variables        = 2
    chi-square         = 4.883
    reduced chi-square = 1.628
[[Variables]]
    A:   1.84022579 +/- 0.198053 (10.76%) (init= 1)
    B:   3.83710261 +/- 0.451325 (11.76%) (init= 1)
[[Correlations]] (unreported correlations are <  0.100)
    C(A, B)                      = -0.932 


In [31]:
((out_bethe_1.best_fit - dE_dx_1)/(delta_dE_dx_1*10)).sum()

8.4006853898789853

In [42]:
plt.plot(np.arange(0,5,0.1),bethe_func(np.arange(0,5,0.1),out_bethe_1.values['A'],out_bethe_1.values['B']),label='Messdaten')
plt.errorbar(E_1,-dE_dx_1,delta_dE_dx_1,fmt='. ',label='Behte-Bloch-Fit')
plt.xlabel(r'E / MeV', fontsize=42)
plt.ylabel(r'$\frac{dE}{dx}$ / MeV $\cdot$ cm$^{-1}$', fontsize=42)
plt.ylim([0,1.9])
plt.xlim([0,2])
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()

In [43]:
out_bethe_2 = bethe_fit.fit(-dE_dx_2,params,x=E_2)
print(out_bethe_2.fit_report())
out_bethe_2.plot()
plt.show()

[[Model]]
    Model(bethe_func)
[[Fit Statistics]]
    # function evals   = 13
    # data points      = 8
    # variables        = 2
    chi-square         = 0.208
    reduced chi-square = 0.035
[[Variables]]
    A:   2.13258856 +/- 0.119300 (5.59%) (init= 1)
    B:   3.24580665 +/- 0.162532 (5.01%) (init= 1)
[[Correlations]] (unreported correlations are <  0.100)
    C(A, B)                      = -0.846 


In [44]:
((out_bethe_2.best_fit - dE_dx_2)/(delta_dE_dx_2*10)).sum()

13.392248379387777

In [45]:
plt.plot(np.arange(0,5,0.1),bethe_func(np.arange(0,5,0.1),out_bethe_2.values['A'],out_bethe_2.values['B']),'b',label='Messdaten')
plt.errorbar(E_2,-dE_dx_2,delta_dE_dx_2,fmt='. g',label='Behte-Bloch-Fit')
plt.xlabel(r'E / MeV', fontsize=42)
plt.ylabel(r'$\frac{dE}{dx}$ / MeV $\cdot$ cm$^{-1}$', fontsize=42)
plt.ylim([0.6,1.9])
plt.xlim([0,3])
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()

In [47]:
out_bethe_3 = bethe_fit.fit(-dE_dx_3,params,x=E_3)
print(out_bethe_3.fit_report())
out_bethe_3.plot()
plt.show()

[[Model]]
    Model(bethe_func)
[[Fit Statistics]]
    # function evals   = 12
    # data points      = 11
    # variables        = 2
    chi-square         = 0.624
    reduced chi-square = 0.069
[[Variables]]
    A:   2.04625746 +/- 0.120395 (5.88%) (init= 1)
    B:   3.53944886 +/- 0.185326 (5.24%) (init= 1)
[[Correlations]] (unreported correlations are <  0.100)
    C(A, B)                      = -0.777 


In [48]:
((out_bethe_3.best_fit - dE_dx_3)/(delta_dE_dx_3*10)).sum()

17.526122503776541

In [49]:
plt.plot(np.arange(0,5,0.1),bethe_func(np.arange(0,5,0.1),out_bethe_3.values['A'],out_bethe_3.values['B']),label='Messdaten')
plt.errorbar(E_3,-dE_dx_3,delta_dE_dx_3,fmt='. ',label='Behte-Bloch-Fit')
plt.xlabel(r'E / MeV', fontsize=42)
plt.ylabel(r'$\frac{dE}{dx}$ / MeV $\cdot$ cm$^{-1}$', fontsize=42)
plt.ylim([0.2,1.9])
plt.xlim([0,3])
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()

In [50]:
out_bethe_4 = bethe_fit.fit(-dE_dx_4,params,x=E_4)
print(out_bethe_4.fit_report())
out_bethe_4.plot()
plt.show()

[[Model]]
    Model(bethe_func)
[[Fit Statistics]]
    # function evals   = 13
    # data points      = 19
    # variables        = 2
    chi-square         = 3.164
    reduced chi-square = 0.186
[[Variables]]
    A:   2.31135448 +/- 0.192054 (8.31%) (init= 1)
    B:   3.22799303 +/- 0.295110 (9.14%) (init= 1)
[[Correlations]] (unreported correlations are <  0.100)
    C(A, B)                      = -0.812 


In [51]:
((out_bethe_4.best_fit - dE_dx_4)/(delta_dE_dx_4*10)).sum()

30.965976560810365

In [53]:
plt.plot(np.arange(0,5,0.1),bethe_func(np.arange(0,5,0.1),out_bethe_4.values['A'],out_bethe_4.values['B']),label='Messdaten')
plt.errorbar(E_4,-dE_dx_4,delta_dE_dx_4,fmt='. ',label='Behte-Bloch-Fit')
plt.ylim([0.2,1.9])
plt.xlim([0,5])
plt.xlabel(r'E / MeV', fontsize=42)
plt.ylabel(r'$\frac{dE}{dx}$ / MeV $\cdot$ cm$^{-1}$', fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()

# Bragg-Kurve

In [55]:
plt.plot(x[:dE_dx_1.size],-dE_dx_1,'o-r')
plt.plot(x[:dE_dx_1.size],-dE_dx_1,'o g',label='Messdaten')
plt.xlabel(r'x / cm', fontsize=42)
plt.ylabel(r'$\frac{dE}{dx}$ / MeV $\cdot$ cm$^{-1}$', fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()

In [56]:
plt.plot(x[:dE_dx_2.size],-dE_dx_2,'o-r')
plt.plot(x[:dE_dx_2.size],-dE_dx_2,'o g',label='Messdaten')
plt.xlabel(r'x / cm', fontsize=42)
plt.ylabel(r'$\frac{dE}{dx}$ / MeV $\cdot$ cm$^{-1}$', fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()

In [57]:
plt.plot(x[:dE_dx_3.size],-dE_dx_3,'o-r')
plt.plot(x[:dE_dx_3.size],-dE_dx_3,'o g',label='Messdaten')
plt.xlabel(r'x / cm', fontsize=42)
plt.ylabel(r'$\frac{dE}{dx}$ / MeV $\cdot$ cm$^{-1}$', fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()

In [58]:
plt.plot(x[:dE_dx_4.size],-dE_dx_4,'o-r')
plt.plot(x[:dE_dx_4.size],-dE_dx_4,'o g',label='Messdaten')
plt.xlabel(r'x / cm', fontsize=42)
plt.ylabel(r'$\frac{dE}{dx}$ / MeV $\cdot$ cm$^{-1}$', fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()

In [59]:
plt.plot(x[:dE_dx_1.size],-dE_dx_1,label='Messdaten')
plt.plot(x[:dE_dx_2.size],-dE_dx_2,label='Messdaten')
plt.plot(x[:dE_dx_3.size],-dE_dx_3,label='Messdaten')
plt.plot(x[:dE_dx_4.size],-dE_dx_4,label='Messdaten')
plt.xlabel(r'x / cm', fontsize=22)
plt.ylabel(r'$\frac{dE}{dx}$ / MeV $\cdot$ cm$^{-1}$', fontsize=22)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='best',fontsize=22)
plt.show()